In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.animation as animation 
import mpl_toolkits.mplot3d.axes3d as p3


In [ ]:
acidities = pd.read_csv("./ass1_data/data/q1/linearX.csv")
densities = pd.read_csv("./ass1_data/data/q1/linearY.csv") 

In [ ]:
def normalize(arr):
    mean = arr.mean()
    # print(mean) 
    # variance = acid_arr.var() 
    std_dev = arr.std()  
    arr = (arr - mean)/std_dev  
    return arr 

In [ ]:

acid_arr = acidities["Acidities"].to_numpy()
acid_arr = normalize(acid_arr) 
# print(acid_arr) 

acidities["Intercept"] = 1 
cols = acidities.columns.to_list() 
cols = cols[1:] + cols[:1] 
acidities = acidities[cols]   
acidities["Acidities"] = acid_arr
# print(acidities) 

density_arr = densities["Densities"].to_numpy() 
density_arr = normalize(density_arr)
densities["Densities"] = density_arr 


In [ ]:
def compute_error(theta, X, Y):
    Z = np.matmul(X, theta) - Y 
    m = Y.size
    return (np.matmul(np.transpose(Z), Z))/(2*m) 

In [ ]:
def compute_gradient(theta , X, Y) : 
    Z = Y - np.matmul(X, theta) 
    # print(Z)
    gradient = np.zeros(theta.size)
    m = Y.size  
    for j in range(theta.size):
        X_j = X[:, j] 
        gradient[j] = np.sum(Z * X_j) / m  
    # gradient = np.sum( Z * X , axis = 0)
    # print(gradient)  
    return gradient 

In [ ]:
dim = 2 
theta = np.zeros(dim) 
X = acidities.to_numpy()  
Y = density_arr 

learning_param = 0.1 

inv = np.linalg.inv(np.matmul(np.transpose(X), X)) 
actual_theta = np.matmul(np.matmul(inv, np.transpose(X)), Y) 

no_of_iterations = 0
# initial_error = compute_error(theta, X, Y) 
# error = initial_error 
# epsilon = 0.0000001 * initial_error 
# prev_error = 2*initial_error # some large value initially   

plot_data = [] 
current_grad = compute_gradient(theta, X, Y) 
epsilon = 0.001 * np.linalg.norm(current_grad) 
while (np.linalg.norm(current_grad) > epsilon):
       # prev_error = error 
       current_grad = compute_gradient(theta, X, Y )
       theta = theta + learning_param * current_grad 
       error = compute_error(theta, X,Y)  
       plot_data.append([theta[0], theta[1], error]) 
       print("error is", error)  
       # print("current theta is", theta ) 
       no_of_iterations += 1 

# print(plot_data)
print("gradient descent result is",theta)
print("actual theta is", actual_theta)  
print(f"no of iterations is {no_of_iterations}") 

## plotting  the data

In [ ]:
plt.scatter(acid_arr, Y, label = "training set") 
plt.title("Training set vs hypothesis function")

H_theta = np.matmul(  acidities, theta ) 
# label = "slope = " + str(theta[1]) + "\n intercept = " + str(theta[0]) 
plt.plot(acid_arr, H_theta, label = "hypothesis function", color = "green") 

plt.xlabel("X axis (normalized acidities)") 
plt.ylabel("Y axis (normalized densities)" )

plt.legend()
plt.show()

## 3D mesh plot of error function

In [ ]:
%matplotlib ipympl 

theta_x = np.arange(-0.5 , 0.5, 0.1 ) 
theta_y = np.arange(0,1 , 0.1) 
X_mesh, Y_mesh = np.meshgrid(theta_x, theta_y) 
print(X_mesh) 
print(Y_mesh)
X_row, X_col = X_mesh.shape
Y_row, Y_col = Y_mesh.shape 

error_surface = np.zeros((X_row, X_col))  
# print(X) 

for i in range(X_row):
    for j in range(X_col):
        x_coord = X_mesh[i,j] 
        y_coord = Y_mesh[i,j] 
        temp_theta = np.array([x_coord, y_coord])   
        error_surface[i,j] = compute_error(temp_theta, X, Y) 


In [ ]:

# Initialize the plot and set up the 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter([], [], [], c='b', marker='.', s = 10, alpha=0.7)

ax.set(xlim3d=(-0.5, 0.5), xlabel='Theta 0')
ax.set(ylim3d=(0, 1), ylabel='Theta 1')
ax.set(zlim3d=(0.2, 0.7), zlabel='Z')

ax.plot_surface(X_mesh, Y_mesh, error_surface, alpha = 0.4)   


# Initialization function
def init():
    sc._offsets3d = ([], [], [])
    return sc,

# Animation function
def update(frame):
    # if (frame % 5 == 0):
    x, y, z = zip(*plot_data[:frame+1])
    sc._offsets3d = (x, y, z)
    return sc,

azim_angle = 6  # Adjust the azimuth angle (horizontal rotation)
elev_angle = 32  # Adjust the elevation angle (vertical rotation)
ax.view_init(elev=elev_angle, azim=azim_angle)

# Create the animation
frame_interval = 200 
ani = animation.FuncAnimation(fig, update, frames=len(plot_data), init_func=init, blit=True, interval = 
                              frame_interval)
ani.save("1_e_animation.png", writer='imagemagick') 
# Display the animation
plt.show()